In [1]:
!pip install --upgrade pip

Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:992)'))) - skipping
Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:992)'))) - skipping


In [2]:
!pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org retry msal openai -q


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
import os
import openai
import requests
import json
from retry import retry
from msal import ClientApplication
import time

In [6]:
@retry (tries=3, delay=2)
def getapikey():
    client_id = os.getenv("AZURE_CLIENT_ID")
    tenant_id = os.getenv("AZURE_TENANT_ID")
    endpoint = os.getenv("AZURE_ENDPOINT")
    scopes = [os.getenv("AZURE_APPLICATION_SCOPE")]
    username=os.getenv("SVC_ACCOUNT")
    password=os.getenv("SVC_PASSWORD")

    app = ClientApplication(
        client_id=client_id,
        authority="https://login.microsoftonline.com/" + tenant_id
    )

    acquire_tokens_result = app.acquire_token_by_username_password(username=username,
        password=password,
        scopes=scopes)

    if 'error' in acquire_tokens_result:
        print("Error: " + acquire_tokens_result['error'])
        print("Description: " + acquire_tokens_result['error_description'])
        return(2)
    else:
        header_token = {"Authorization": "Bearer {}".format(acquire_tokens_result['access_token'])}
        rt = requests.post(url=endpoint, headers=header_token, data=b'{"key":"openaikey2"}')
        return(rt.json())

In [7]:
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
# We are dynamically getting the key from AZURE.  Access is based on the service account/ad group combination
openai.api_key = getapikey()

RuntimeError: WsTrust server returned error in RSTR: {'reason': 'Authentication failed', 'code': 'wst:FailedAuthentication'}